In [1]:
import os 
from openai import AzureOpenAI
from dotenv import load_dotenv

In [2]:
# Initialize the Azure OpenAI client
load_dotenv()
 # Initialize the Azure OpenAI client
client = AzureOpenAI(
     azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
     api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
     api_version=os.getenv("AZURE_OPENAI_API_VERSION")
 )

In [3]:
# Function description for the model to read
tools = [
     {
         "type": "function",
         "function": {
             "name": "get_current_time",
             "description": "Get the current time in a given location",
             "parameters": {
                 "type": "object",
                 "properties": {
                     "location": {
                         "type": "string",
                         "description": "The city name, e.g. San Francisco",
                     },
                 },
                 "required": ["location"],
             },
         }
     },
     {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco",
                        },
                    },
                    "required": ["location"],
                },
            }
     }
 ]

In [7]:
# Initial user message
messages = [{"role": "user", "content": "What's the current time in San Francisco"}] 
# messages = [{"role": "user", "content": "What's the weather like in San Francisco?"}]  
 # First API call: Ask the model to use the function
response = client.chat.completions.create(
       model="gpt-4.1",
       messages=messages,
       tools=tools,
       tool_choice="auto",
   )
  
   # Process the model's response
response_message = response.choices[0].message
messages.append(response_message)
  
print("Model's response:")  

print(response_message)

Model's response:
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zx7EIwmhPiEKgwRN0InoO57F', function=Function(arguments='{"location":"San Francisco"}', name='get_current_time'), type='function')])


In [8]:
from zoneinfo import ZoneInfo
import json
from datetime import datetime

# Define timezone data mapping
TIMEZONE_DATA = {
    "san francisco": "America/Los_Angeles",
    "new york": "America/New_York",
    "london": "Europe/London",
    "tokyo": "Asia/Tokyo",
    "sydney": "Australia/Sydney"
}

def get_current_time(location):
     """Get the current time for a given location"""
     print(f"get_current_time called with location: {location}")  
     location_lower = location.lower()
        
     for key, timezone in TIMEZONE_DATA.items():
         if key in location_lower:
             print(f"Timezone found for {key}")  
             current_time = datetime.now(ZoneInfo(timezone)).strftime("%I:%M %p")
             return json.dumps({
                 "location": location,
                 "current_time": current_time
             })
      
     print(f"No timezone data found for {location_lower}")  
     return json.dumps({"location": location, "current_time": "unknown"})
 

def get_weather(location):
     """Get the current weather for a given location"""
     print(f"get_weather called with location: {location}")  
     location_lower = location.lower()
        
     for key, timezone in TIMEZONE_DATA.items():
         if key in location_lower:
             print(f"Timezone found for {key}")  
             return json.dumps({
                 "location": location,
                 "weather": "sunny"
             })
      
     print(f"No timezone data found for {location_lower}")  
     return json.dumps({"location": location, "weather": "unknown"})

In [9]:
def handle_tool_calls(response_message, messages):
    # Handle function calls
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            if tool_call.function.name == "get_current_time":
                function_args = json.loads(tool_call.function.arguments)
                time_response = get_current_time(
                    location=function_args.get("location")
                )
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "get_current_time",
                    "content": time_response,
                })
            elif tool_call.function.name == "get_weather":
                function_args = json.loads(tool_call.function.arguments)
                weather_response = get_weather(
                    location=function_args.get("location")
                )
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "get_weather",
                    "content": weather_response,
                })
    else:
        print("No tool calls were made by the model.")  

    # Second API call: Get the final response from the model
    final_response = client.chat.completions.create(
        model="gpt-4.1",
        messages=messages,
    )
    return final_response.choices[0].message.content

# Call the function and print the result
result = handle_tool_calls(response_message, messages)
print(result)

get_current_time called with location: San Francisco
Timezone found for san francisco
The current time in San Francisco is 2:18 AM.
